In [ ]:
from os.path import exists
from urllib.request import urlretrieve
import geopandas as gpd
import pandas as pd
import hvplot.pandas

from holoviews import opts
opts.defaults(opts.Overlay(active_tools=['wheel_zoom'], frame_width=500, frame_height=400))

import warnings
warnings.filterwarnings('ignore')

def get_gdf_from_wfs(layer):
    """
    Get geopandas.GeoDataFrame from data.wien.gv.at WFS service based on layer name
    
    Parameters
    ----------
    layer : string
        WFS layer name 
    """
    file = f'{layer}.json'
    url = f"https://data.wien.gv.at/daten/geo?service=WFS&request=GetFeature&version=1.1.0&typeName=ogdwien:{layer}&srsName=EPSG:4326&outputFormat=json"
    if not exists(file):
        try:
            urlretrieve(url, file)
        except:
            from pyodide.http import open_url
            url_contents = open_url(url)
            file = url_contents.read()
    return gpd.read_file(file)

def plot_with_buffer(gdf, buffer_size, *args, **kwargs):
    """
    Convenience function for plotting a GeoPandas point GeoDataFrame using point markers plus buffer polygons
    
    Parameters
    ----------
    gdf : geopandas.GeoDataFrame
        point GeoDataFrame to plot
    buffer_size : numeric
        size of the buffer in meters (measured in EPSG:31287)
    """
    buffered = gdf.to_crs('epsg:31287').buffer(buffer_size)
    buffered = gdf.copy().set_geometry(buffered).to_crs('epsg:4326')
    
    plot = ( buffered.hvplot(geo=True, tiles='OSM', alpha=0.5, line_width=0, *args, **kwargs) * 
      gdf.hvplot(geo=True, hover_cols='all') 
    ).opts(active_tools=['wheel_zoom'])
    
    #plot = buffered.explore(height=500)
    #plot = gdf.explore(m=plot)
    
    return plot


In [ ]:
import panel as pn

pn.extension(sizing_mode="stretch_width", template="fast")
pn.state.template.param.update(site="Panel in the Browser", title="Data.Wien.gv.at Viewer")

wfs_layer = pn.widgets.TextInput(name="WFS Layer (e.g. RADGRAETZELOGD, SCOOTERABSTELLOGD, ...): ", 
                                 value="RADGRAETZELOGD")
buffer_size = pn.widgets.IntSlider(start=100, end=1000, name="Buffer size (meters)")

def pipeline(wfs_layer, buffer_size):
    gdf = get_gdf_from_wfs(wfs_layer)
    return plot_with_buffer(gdf, buffer_size)

pn.Column(
    "For a full list of WFS layers, see https://www.data.gv.at/auftritte/?organisation=stadt-wien&formatFilter%5B0%5D=WFS",
    wfs_layer,
    "Move the slider below to change the buffer size",
    buffer_size,
    pn.bind(pipeline, wfs_layer, buffer_size),
).servable()